# Pipeline Prototype

In [13]:
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasRegressor
from models.tf.models import LongShortTermMemory
from models.tf.dataset import StockDataGenerator
from sklearn.model_selection import KFold, GridSearchCV
from joblib import Memory

In [14]:
VERBOSITY = 1

In [15]:
feature_combos = { 
    'set1': {}, 
    'set2':
    { 
        'EMA': {}, '%B': {}, 'CCI': {}, 
        'RSI': {}, 'VIX': {}, 'MIDPRICE': {}, 
        'MAMA': {}, 'ROCR100': {}
    },
    'set3':
    {
        'EMA': {}, '%B': {'timeperiod': 15}, 'CCI': {}, 
        'RSI': {}, 'VIX': {}, 'MIDPRICE': {'timeperiod': 15},
        'MAMA': {}, 'ROCR100': {}, 'AROONOSC': {}
    }
}

In [16]:
model_1 = LongShortTermMemory()
models = [ model_1 ]
datasets = []
for k, v in feature_combos.items():
    datasets.append( StockDataGenerator(
            'BLK', api='TDA',
            period='FIVE_YEAR', period_type='YEAR', 
            frequency='DAILY', frequency_type='DAILY',
            features=v, verbose=0, target='close', 
            lookback=0, pc=False, 
    ))
for model in models:
    for data in datasets:
        lstm = KerasRegressor(model=model.compile_model(data.X_train, name=f'set_{k}',verbose=VERBOSITY))
        model_pipeline = Pipeline([
            (f'lstm_feature_{lstm.__name__}', lstm)
        ])
        model_params = {
            'lstm__epochs': 25, 'lstm__verbose': 1
        }

model_pipeline.fit(data.X_train, data.y_train, **model_params)

    

Model: "set_set3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_64 (LSTM)              (None, 5, 100)            40800     
                                                                 
 dropout_64 (Dropout)        (None, 5, 100)            0         
                                                                 
 lstm_65 (LSTM)              (None, 5, 50)             30200     
                                                                 
 dropout_65 (Dropout)        (None, 5, 50)             0         
                                                                 
 bidirectional_32 (Bidirecti  (None, 5, 100)           40400     
 onal)                                                           
                                                                 
 dropout_66 (Dropout)        (None, 5, 100)            0         
                                                          

KeyError: 'lstm'